# Случайные леса
__Суммарное количество баллов: 10__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [55]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [56]:
from task import gini, entropy, gain

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [57]:
from task import DecisionTree

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [58]:
class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        raise NotImplementedError()

    
    def fit(self, X, y):
        raise NotImplementedError()
    
    def predict(self, X):
        raise NotImplementedError()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [59]:
def feature_importance(rfc):
    raise NotImplementedError()

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [60]:
from task import RandomForestClassifier
from task import feature_importance, out_of_bag_error, most_important_features

In [61]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))

/home/roman/edu/ML/Homework 9/task.py:187: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  # X, y = synthetic_dataset(10)
/home/roman/edu/ML/Homework 9/task.py:225: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  err_oob=out_of_bag_error(rfc, rfc.X, rfc.y)


Accuracy: 1.0
Importance: [0.0, 0.0, 0.20599999999999996, 0.19599999999999995, 0.44699999999999995, 0.0]


### Задание 4 (3 балла)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [62]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [63]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [64]:
from task import rfc_age

rfc_age = RandomForestClassifier(n_estimators=10, max_depth=20, max_features=5, criterion="gini")
rfc_age.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc_age.predict(X_test) == y_age_test))
# print("Most important features:")
# for i, name in enumerate(most_important_features(feature_importance(rfc_age), features, 20)):
#     print(str(i+1) + ".", name)

Accuracy: 0.6532156368221942


/home/roman/edu/ML/Homework 9/task.py:187: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  # X, y = synthetic_dataset(10)
/home/roman/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:112: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  warnings.warn("The input array could not be properly "


#### Пол

In [65]:
from task import rfc_gender

rfc_gender = RandomForestClassifier(n_estimators=10)
rfc_gender = RandomForestClassifier(n_estimators=5, max_depth=10, max_features=5, criterion="gini")
rfc_gender.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc_gender.predict(X_test) == y_sex_test))
# print("Most important features:")
# for i, name in enumerate(most_important_features(feature_importance(rfc_gender), features, 20)):
#     print(str(i+1) + ".", name)

Accuracy: 0.7704918032786885


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [66]:
X, y = synthetic_dataset(1000)
cat=CatBoostClassifier(iterations=100,  loss_function='MultiClass')
cat.fit(X,y,verbose=False)
print("Accuracy:", np.mean(cat.predict(X).ravel() == y))
print("Importance:", cat.get_feature_importance())

Accuracy: 1.0
Importance: [1.08654747e-03 1.37258052e-02 2.77976477e+01 2.78738495e+01
 4.43096880e+01 4.00247088e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

Оценка:
1. 1 балл за исправно работающий код
2. +1 балл за точность предсказания возростной группы выше 65%
3. +1 балл за точность предсказания пола выше 75%

In [67]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

#### Возраст

In [81]:
cat_age=CatBoostClassifier(iterations=100,loss_function='MultiClass')
cat_age.fit(X_train,y_age_train,verbose=False)
print("Accuracy:", np.mean(cat_age.predict(X_test).ravel() == y_age_test))

cat_age.save_model('cat_age.cbm')

#print("Most important features:")
#for i, name in enumerate(most_important_features(None, features, 10)):
#    print(str(i+1) + ".", name)

Accuracy: 0.7389659520807061


#### Пол

In [82]:
cat_sex=CatBoostClassifier(iterations=100,loss_function='MultiClass')
cat_sex.fit(X_train,y_sex_train,verbose=False)
print("Accuracy:", np.mean(cat_sex.predict(X_test).ravel() == y_sex_test))

cat_sex.save_model('cat_sex.cbm')
#print("Accuracy:", np.mean(None == y_sex_test))
# print("Most important features:")
# for i, name in enumerate(most_important_features(None, features, 10)):
#     print(str(i+1) + ".", name)

Accuracy: 0.8776796973518285
